# Análisis de Cobertura de GloVe

En este notebook se analiza la cobertura del vocabulario del corpus de noticias respecto al modelo preentrenado GloVe. Se evaluará qué porcentaje de palabras del corpus están presentes en GloVe.

Importar todas las librerías necesarias

In [ ]:
import os
from collections import Counter
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk

Descargar recursos necesarios de NLTK

In [ ]:
nltk.download('punkt')

Definir directorios de donde queremos cargar los datos resumidos y la ubicación del modelo GloVe

In [ ]:
# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos
SUMMARIES_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'summarized')
GLOVE_PATH = os.path.join(ROOT_DIR, 'models', 'glove.2024.wikigiga.300d.txt')

print(f"Directorio de resúmenes: {SUMMARIES_DIR}")
print(f"Ruta de GloVe: {GLOVE_PATH}")

Cargar dataset de ejemplo para análisis

In [ ]:
# Cargar dataset
df = pd.read_parquet(os.path.join(SUMMARIES_DIR, 'nvidia_news.parquet'), engine="pyarrow")

# Concatenar título y resumen
df["text"] = df["title"] + "\n\n" + df["body_summary"]
df = df.drop(columns=["clean_body", "body_summary", "teaser", "title", "author"])

print(f"Dataset cargado con {len(df)} registros")
print(f"Primeras filas:")
df.head()

Tokenizar todo el texto del corpus y construir vocabulario

In [ ]:
# Tokenizar todo el texto
all_words = []
for text in df['text']:
    tokens = word_tokenize(text.lower())
    all_words.extend(tokens)

# Contar frecuencias
word_counts = Counter(all_words)
unique_words = set(word_counts.keys())

print(f"Total de palabras (tokens): {len(all_words):,}")
print(f"Palabras únicas: {len(unique_words):,}")

Cargar vocabulario de GloVe

In [ ]:
# Cargar vocabulario de GloVe (solo palabras, no vectores)
glove_vocab = set()
with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        word = line.split(' ')[0]  # Primer elemento es la palabra
        glove_vocab.add(word)

print(f"Vocabulario de GloVe cargado: {len(glove_vocab):,} palabras")

Comparar vocabulario del corpus con GloVe y calcular cobertura

In [ ]:
# Comparar vocabularios
words_in_glove = unique_words & glove_vocab
words_not_in_glove = unique_words - glove_vocab

# Calcular cobertura por tokens (considerando frecuencias)
total_words = len(all_words)
tokens_not_in_glove = sum(count for word, count in word_counts.items() if word not in glove_vocab)
token_coverage = ((total_words - tokens_not_in_glove) / total_words) * 100

# Mostrar resultados
print(f"Total de palabras en corpus: {total_words:,}")
print(f"Palabras únicas en corpus: {len(unique_words):,}")
print(f"Palabras únicas en GloVe: {len(words_in_glove):,}")
print(f"Palabras únicas NO en GloVe: {len(words_not_in_glove):,}")
print(f"Cobertura de tokens (por frecuencia): {token_coverage:.2f}%")